In [1]:
import h5py
import numpy as np
import pandas as pd
import tables
import os
import sys
import time
import glob
import scipy.io as sio
import hdf5_getters
from pandas import read_hdf

In [ ]:
### Need to include track_id in pickle file #######
finalWorkingDF = pd.read_pickle("finalWorkingDF")
#finalWorkingDF.to_pickle("musicMatchDB")
finalWorkingDF["words"] = np.nan

In [19]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

database = "mxm_dataset.db"
conn = create_connection(database)
cur = conn.cursor()

cur.execute("SELECT * FROM lyrics")
# count = 0
# for row in cur:
#     print(row)
#     count += 1
#     if count==600:
#         break

# print("break")

corpus = []
currID = "bogus"
#vectorizer = CountVectorizer()
count = 0
newStr = ""
for row in cur:
    if currID != row[0]:
        # Add newDict to SKT word thing
        if count != 0:
            corpus.append(newStr[0:len(newStr)-1])
        newStr = row[3]*(row[2]+" ")
        currID = row[0]
        count += 1
    else:
        newStr += row[3]*(row[2]+" ")
        
    if count == 10:
        break

        
#X = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())
#print(X.toarray())

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(corpus)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

# see final the clean data
print (train_data_features.shape)
    
vocab = vectorizer.get_feature_names()


from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

(9, 481)
['about', 'adam', 'ador', 'advic', 'afraid', 'after', 'again', 'ai', 'air', 'all', 'along', 'alreadi', 'alt', 'am', 'an', 'and', 'angel', 'anoth', 'apart', 'are', 'around', 'arrang', 'as', 'ash', 'ask', 'asleep', 'at', 'aw', 'away', 'back', 'be', 'beach', 'beat', 'been', 'befor', 'believ', 'bend', 'best', 'better', 'bid', 'big', 'bite', 'black', 'bless', 'block', 'bodi', 'bone', 'boss', 'boy', 'breath', 'bridg', 'bright', 'buck', 'burn', 'burnt', 'busi', 'but', 'by', 'ca', 'came', 'can', 'captur', 'car', 'care', 'cast', 'catch', 'caus', 'celebr', 'chanc', 'chang', 'check', 'choos', 'clay', 'clear', 'clock', 'close', 'coast', 'colder', 'color', 'come', 'concern', 'cos', 'could', 'cover', 'cri', 'curl', 'damn', 'dare', 'day', 'de', 'debt', 'deep', 'delight', 'devast', 'die', 'differ', 'din', 'direct', 'dissolv', 'do', 'doe', 'done', 'door', 'doubl', 'down', 'dream', 'drop', 'du', 'each', 'echo', 'edg', 'ei', 'element', 'em', 'en', 'end', 'eras', 'et', 'everi', 'everyth', 'exist'

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}
(1, 8)
<class 'scipy.sparse.csr.csr_matrix'>
[[1 1 1 1 1 1 1 2]]
{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer()
vect

# use TreeankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
vect.set_params(tokenizer=tokenizer.tokenize)

# remove English stop words
vect.set_params(stop_words='english')

# include 1-grams and 2-grams
vect.set_params(ngram_range=(1, 2))

# ignore terms that appear in more than 50% of the documents
vect.set_params(max_df=0.5)

# only keep terms that appear in at least 2 documents
vect.set_params(min_df=2)

In [ ]:
# data.merge(genres,on='track_id')
usedSongLyrics = pd.DataFrame(columns=['TrackID','MusicMatchID','Word','Count','Training/Testing'])

count = 0
for row in cur:
    if row[0] in trackIDs:
        usedSongLyrics.loc[count] = row
        count += 1


